## Extracting the Coordinates

In [1]:
import requests
import json

In [2]:
# Overpass API endpoint
url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"

In [3]:
# Example query: All cycling routes in Europe (bicycle routes)
#-----------------------------------------------CHANGE COORDS HERE--------------------------------------#
query = """
[out:json][timeout:900];
(
  relation["route"="bicycle"](49.9,-7.6,58.7,1.8);
);
out geom;
"""
#-----------------------------------------------CHANGE COORDS HERE--------------------------------------#

In [4]:
# Make the request
response = requests.get(url, params={'data': query})

In [5]:
# Check success
if response.status_code == 200:
    data = response.json()
    print("✅ Success:", len(data['elements']), "routes found")
else:
    print("❌ Error:", response.status_code)

✅ Success: 8318 routes found


In [6]:
# Save response to file
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
with open("UK_cycle_routes.json", "w") as f:
    json.dump(data, f)
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [7]:
import pandas as pd

# Create empty list to store results
routes_summary = []

# Loop through each route in the dataset
for route in data['elements']:
    if route['type'] != 'relation':
        continue  # skip non-route elements

    coords = []
    for member in route.get('members', []):
        if 'geometry' in member:
            segment_coords = [(pt['lon'], pt['lat']) for pt in member['geometry']]
            coords.extend(segment_coords)

    if coords:
        start_coord = coords[0]
        end_coord = coords[-1]
        routes_summary.append({
            'id': route.get('id'),
            'name': route.get('tags', {}).get('name', 'Unnamed route'),
            'start_lon': start_coord[0],
            'start_lat': start_coord[1],
            'end_lon': end_coord[0],
            'end_lat': end_coord[1],
            'num_points': len(coords)
        })

In [8]:
# Convert results to DataFrame
routes_df = pd.DataFrame(routes_summary)

In [9]:
routes_df = routes_df.sort_values(by='num_points', ascending=False)

In [10]:
routes_df

,id,name,start_lon,start_lat,end_lon,end_lat,num_points
24,12461,Unnamed route,-1.193491,52.961078,-0.834117,52.120161,30595
19,11017,Unnamed route,-4.225682,57.477388,-4.695980,55.656862,28727
2624,9540148,EuroVelo 1 - Atlantic Coast Route - part Unite...,-4.985964,52.009915,-2.999010,51.123030,28171
670,2763769,EuroVelo 1 - Atlantic Coast Route - part Unite...,-2.189547,57.205800,-5.036523,54.984898,22762
22,11237,Pennine Cycleway,-1.992570,53.365548,-2.200559,53.854031,19898
...,...,...,...,...,...,...,...
4889,13613219,LCN 23,-0.098567,51.363778,-0.098581,51.363814,2
4954,13874835,Unnamed route,0.002450,51.513863,0.002505,51.513839,2
7117,18318988,Unnamed route,0.075856,51.562223,0.075233,51.562967,2
7860,19254024,Royal Canal Greenway,-6.369882,53.381855,-6.370297,53.381792,2


# Extracting the features UK First 2,000

In [ ]:
import os

AK = os.getenv("AK")

In [12]:
import openrouteservice
import pandas as pd
import json
import time
import os

In [13]:
# Initialize client
client = openrouteservice.Client(key=AK)  # Replace AK with your ORS API key

In [14]:
results = []
save_interval = 10

In [15]:
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
save_path = "ors_route_results_UK2.json"
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [16]:
test_df = routes_df.iloc[2000:4000]

In [17]:
test_df

,id,name,start_lon,start_lat,end_lon,end_lat,num_points
3435,11051917,Unnamed route,-0.089943,51.520748,-0.093004,51.510944,215
2318,9049069,Route M,-0.207064,51.127957,-0.189533,51.116674,215
4292,12110850,LCN 29,-0.196534,51.440400,-0.209986,51.420409,215
3915,11538083,A56 Interim Scheme,-2.313462,53.438898,-2.311021,53.442603,215
3376,10926812,Crab & Winkle Link,1.077979,51.286274,1.053935,51.298225,214
...,...,...,...,...,...,...,...
7311,18737408,Unnamed route,-6.666658,53.217563,-6.673658,53.241009,78
1524,8462830,Enfield local cycle network,-0.030742,51.648618,-0.058449,51.622688,78
7323,18769579,Unnamed route,-6.696187,53.664506,-6.680470,53.667735,78
3923,11554362,Unnamed route,-5.906305,54.601313,-5.900296,54.601605,78


In [ ]:
for i, row in test_df.iterrows():
    start = (row['start_lon'], row['start_lat'])
    end = (row['end_lon'], row['end_lat'])

    try:
        route = client.directions(
            coordinates=[start, end],
            profile='cycling-regular',
            format='geojson',
            elevation=True,
            instructions=True,
            extra_info=['surface', 'waytype', 'waycategory', 'steepness'],
        )



        # --- SAVE RAW ORS RESPONSE FOR DEBUGGING ---
        #-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
        raw_dir = "raw_ors_responses_UK2"
        #-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
        os.makedirs(raw_dir, exist_ok=True)

        raw_filename = f"{raw_dir}/route_{row['id']}.json"

        with open(raw_filename, "w") as raw_file:
            json.dump(route, raw_file, indent=2)
        # -------------------------------------------


        props = route['features'][0]['properties']
        summary = props['summary']
        extras = props.get('extras', {})
        segments = props['segments']
        steps = segments[0]['steps']
        turn_steps = [s for s in steps if s['type'] in {0,1,2,3,4,5,6,7}]
        turns = len(turn_steps)
        steps = len(steps)
    

        results.append({
            'id': row['id'],
            'name': row['name'],
            'distance_m': summary.get('distance'),
            'duration_s': summary.get('duration'),
            'ascent_m': [props['ascent']],
            'descent_m': [props['descent']],
            'steps': steps,
            'turns': turns,
            'surface': extras.get('surface', {}).get('values', []),
            'waytype': extras.get('waytype', {}).get('values', []),
            'waycategory': extras.get('waycategory', {}).get('values', []),
            'steepness': extras.get('steepness', {}).get('values', []),
        })

        print(f"✅ Route {i+1} processed successfully")
        time.sleep(1.5)  # rate-limit protection

        # Save partial file
        if (i + 1) % save_interval == 0:
            with open(save_path, "w") as f:
                json.dump(results, f, indent=2)
            print(f"💾 Saved partial results after {i+1} routes")

    except Exception as e:
        print(f"❌ Error on route {row['id']}: {e}")
        continue

✅ Route 3436 processed successfully
✅ Route 2319 processed successfully
✅ Route 4293 processed successfully
✅ Route 3916 processed successfully
✅ Route 3377 processed successfully
✅ Route 4095 processed successfully
✅ Route 7202 processed successfully
✅ Route 4460 processed successfully
💾 Saved partial results after 4460 routes
✅ Route 2283 processed successfully
✅ Route 4505 processed successfully
✅ Route 3394 processed successfully
✅ Route 7944 processed successfully
✅ Route 7887 processed successfully
✅ Route 7000 processed successfully
💾 Saved partial results after 7000 routes
✅ Route 3606 processed successfully
✅ Route 116 processed successfully
✅ Route 6032 processed successfully
✅ Route 905 processed successfully
✅ Route 7622 processed successfully
✅ Route 2159 processed successfully
✅ Route 4218 processed successfully
✅ Route 8242 processed successfully
✅ Route 5105 processed successfully
✅ Route 5217 processed successfully
✅ Route 6143 processed successfully
✅ Route 8127 proce

In [19]:
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
final_path = "ors_route_results_UK_2.json"
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [20]:
with open(final_path, "w") as f:
    json.dump(results, f, indent=2)

In [21]:
print(f"\n🎉 Finished run — processed {len(results)} routes.")
print(f"📁 Full output saved to {final_path}")


🎉 Finished run — processed 1999 routes.
📁 Full output saved to ors_route_results_UK_2.json


In [22]:
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
with open("ors_route_results_UK_2.json", "r") as f:
    results = json.load(f)
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [23]:
# Convert to a DataFrame
df = pd.DataFrame(results)

In [24]:
# Display the first few rows
display(df.head(1))

,id,name,distance_m,duration_s,ascent_m,descent_m,steps,turns,surface,waytype,waycategory,steepness
0,11051917,Unnamed route,1463.7,319.2,[31.0],[50.0],16,14,"[[0, 5, 3], [5, 15, 0], [15, 33, 3], [33, 35, ...","[[0, 16, 2], [16, 30, 3], [30, 31, 2], [31, 33...","[[0, 90, 0]]","[[0, 90, -1]]"


In [25]:
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#
df.to_csv("UK2.csv", index=False)
#-----------------------------------------------CHANGE FILE NAME HERE--------------------------------------#

In [26]:
df

,id,name,distance_m,duration_s,ascent_m,descent_m,steps,turns,surface,waytype,waycategory,steepness
0,11051917,Unnamed route,1463.7,319.2,[31.0],[50.0],16,14,"[[0, 5, 3], [5, 15, 0], [15, 33, 3], [33, 35, ...","[[0, 16, 2], [16, 30, 3], [30, 31, 2], [31, 33...","[[0, 90, 0]]","[[0, 90, -1]]"
1,9049069,Route M,1985.8,397.1,[12.0],[5.0],11,9,"[[0, 51, 3], [51, 52, 0], [52, 72, 3]]","[[0, 21, 6], [21, 22, 3], [22, 34, 2], [34, 40...","[[0, 72, 0]]","[[0, 72, 0]]"
2,12110850,LCN 29,3064.0,614.3,[51.1],[38.1],19,16,"[[0, 1, 3], [1, 2, 14], [2, 4, 3], [4, 7, 14],...","[[0, 1, 6], [1, 2, 7], [2, 4, 6], [4, 7, 7], [...","[[0, 92, 0]]","[[0, 53, 1], [53, 92, -1]]"
3,11538083,A56 Interim Scheme,841.7,173.5,[7.1],[2.1],10,5,"[[0, 1, 0], [1, 21, 3], [21, 27, 0], [27, 49, 3]]","[[0, 1, 1], [1, 22, 6], [22, 27, 3], [27, 31, ...","[[0, 49, 0]]","[[0, 49, 0]]"
4,10926812,Crab & Winkle Link,2779.4,601.0,[70.4],[9.4],18,13,"[[0, 49, 3], [49, 57, 0], [57, 101, 3]]","[[0, 7, 7], [7, 13, 6], [13, 14, 2], [14, 23, ...","[[0, 101, 0]]","[[0, 38, 1], [38, 41, 3], [41, 51, 2], [51, 10..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1994,18737408,Unnamed route,2702.7,576.4,[4.4],[16.4],5,3,"[[0, 7, 3], [7, 8, 0], [8, 20, 3], [20, 22, 0]...","[[0, 7, 3], [7, 43, 2]]","[[0, 43, 0]]","[[0, 43, 0]]"
1995,8462830,Enfield local cycle network,4286.1,865.1,[24.8],[24.8],23,20,"[[0, 4, 3], [4, 9, 0], [9, 68, 3], [68, 69, 0]...","[[0, 4, 6], [4, 6, 7], [6, 7, 6], [7, 9, 7], [...","[[0, 117, 0]]","[[0, 117, 0]]"
1996,18769579,Unnamed route,1200.8,240.1,[9.1],[4.1],4,2,"[[0, 16, 4], [16, 18, 3], [18, 34, 4]]","[[0, 34, 6]]","[[0, 34, 0]]","[[0, 34, 0]]"
1997,11554362,Unnamed route,403.8,80.8,[2.0],[3.0],2,0,"[[0, 17, 3]]","[[0, 17, 2]]","[[0, 17, 0]]","[[0, 17, 0]]"
